# 제4 고지 : 신경망 만들기 
## STEP 38 : 형상 변환 함수

이전 단계에서는 텐서를 사용한 원소별 연산에 대해 살펴봤다. 이번엔 **원소별로 계산하지 않는 함수**에 대해 살펴볼텐데, 그 첫걸음으로 다음 두가지 함수를 구현한다.

1. `reshape` 함수
2. `transpose` 함수


### 38.1 reshape 함수 구현

구현하기 앞서 넘파이의 reshape 사용법을 살펴보면 다음과 같다.

```python
import numpy as np 

x = np.array([[1,2,3],[4,5,6]])
y = np.reshape(x,(6,))
print(y)
# [1 2 3 4 5 6]
```

위의 코드에서 알 수 있듯이 **원소 수는 같고 형상만 바뀐다.**  이제 이와 같이 DeZero를 구현할텐데, 문제는 **역전파를 어떻게 계산할지** 이다.  
(**계산을 원소별로 하지 않는 함수는 텐서의 형상을 고려**해야 한다.)

앞서 살펴봤듯이 `reshape` 함수는 단순히 형상만 변환하며 구체적인 계산은 하지 않는다. 따라서 **역전파는 출력쪽에서 전해지는 기울기를 단순히 입력쪽으로만 흘려보내준다.**  
이때, 다음 그림과 같이 **기울기의 형상이 형상을 변환하기전의 입력의 형상과 같도록 변환하여 흘려보내준다.**


![img](../assets/%EA%B7%B8%EB%A6%BC%2038-1.png)

구체적으로 다음 과정을 따를 수 있도록 한다.

1. `Reshape` 클래스를 초기화할때 변형 목표가 되는 형상 `shape` 정보를 가지고 있는다.
2. `forward` 시 넘파이의 `reshape` 함수를 사용하여 형상을 변환한다. 이때, `self.x_shape = x.shape`로 입력의 형상을 기억해둔다.
3. `backward` 시 기억해둔 `self.x_shape` 를 바탕으로 **기울기 형상이 형상을 변환하기전의 입력의 형상과 같도록 변환**하여 흘려보낸다.

```python
class Reshape(Function):
    def __init__(self,shape) -> None:
        self.shape = shape 
        
    def forward(self,x):
        self.x_shape = x.shape #
        y = x.reshape(self.shape)
        return y 
    
    def backward(self, gy):
        return reshape(gy,self.x_shape)
    
def reshape(x,shape):
    # x : ndarray or Variable
    if x.shape == shape:
        # ndarray 인 경우 variable
        return as_variable(x)
    return Reshape(shape)(x)
```

함수를 구현함에 있어 꼭 기억해야할 것은 DeZero함수는 **항상 `Variable`  또는 `ndarray` 인스턴스를 입력받아 `Variable` 인스턴스를 반환**한다는 것이다.  
`Function` 클래스를 상속한 함수라면 `__call__` 메서드에서 `ndarray` 인스턴스는 자동으로 `Variable` 인스턴스로 변환된다.

예제를 통해 실제로 어떻게 데이터가 흐르는지 확인해보면 다음 그림과 같다.

![img](../assets/%EA%B7%B8%EB%A6%BC%2038-2.png)


In [1]:
import sys 
sys.path.append("..")

import numpy as np 
from dezero import Variable
import dezero.functions as F 

x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.reshape(x,(6,))
y.backward(retain_grad=True)
print(x.grad)



variable([[1 1 1]
          [1 1 1]])


### 38.2 Variable에서 reshape 사용하기

이제는 `reshape` 함수를 더 편하게 만들기 위해서 다음 예시의 넘파이의 `reshape` 와 비슷하게 만드는 작업을 한다.

```python
import numpy as np 
x = np.random.rand(1,2,3) # 1 x 2 x 3 

y = x.reshape((2,3)) # 튜플로 받기
y = x.reshape([2,3]) # 리스트로 받기
y = x.reshape(2,3) # 인수 그대로 풀어서 받기 
```

이를 위해 `Variable` 클래스에 다음 코드를 추가한다.
```python
import dezero
class Variable:
    ...
    def reshape(self, *shape):
        if len(shape) == 1 and isinstance(shape[0], (tuple, list)):
            shape = shape[0]
        return dezero.functions.reshape(self, shape)  # 순환 임포트를 피하기 위해 F.reshape 로 작성하지 않는다.
```


In [2]:
x = Variable(np.random.randn(1,2,3))
print(x)
print("="*50)
y= x.reshape((2,3))
print(y)
y = x.reshape(2,3)
print(y)

variable([[[ 0.89794014 -1.11045713 -0.05856688]
           [ 1.22089265 -0.54260328  0.61546125]]])
variable([[ 0.89794014 -1.11045713 -0.05856688]
          [ 1.22089265 -0.54260328  0.61546125]])
variable([[ 0.89794014 -1.11045713 -0.05856688]
          [ 1.22089265 -0.54260328  0.61546125]])


### 38.3 행렬의 전치 

이어서 다음 그림과 같이 행렬의 전치 함수를 구현한다.

![img](../assets/%EA%B7%B8%EB%A6%BC%2038-3.png)

전치함수 역시 넘파이에서 사용가능하며, `reshape` 함수와 같이 단순히 기울기의 형상만 순전파때와 **'반대'** 형태로 변환하여 구현한다.

```python
class Transpose(Function):
    def forward(self,x):
        y = np.transpose(x)
        return y 
    
    def backward(self, gy):
        gx = transpose(gy)
        return gx 
    
def transpose(x):
    return Transpose()(x)
```

In [3]:
import numpy as np 
from dezero import Variable
import dezero.functions as F 

x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.transpose(x)
y.backward()
print(x.grad)



variable([[1 1 1]
          [1 1 1]])


또한 `Variable` 인스턴스에서도 transpose를 활용할 수 있도록 다음 코드를 추가한다.
```python
import dezero
class Variable:
    ...
    def transpose(self):
        return dezero.functions.transpose(self)

    @property
    def T(self):
        return dezero.functions.transpose(self)
```

In [4]:
x = Variable(np.random.rand(2,3))
print(x)
print("="*50)
y = x.transpose()
print(y)
y = x.T
print(y)

variable([[0.61362762 0.17949527 0.22263542]
          [0.07592891 0.10039808 0.54400071]])
variable([[0.61362762 0.07592891]
          [0.17949527 0.10039808]
          [0.22263542 0.54400071]])
variable([[0.61362762 0.07592891]
          [0.17949527 0.10039808]
          [0.22263542 0.54400071]])


### 38.4 [보충] 실제 transpose 함수

넘파이의 `np.transpose` 는 좀 더 범용적으로 활용할 수 있는데, 예를 들어 다음과 같이 **축의 데이터 순서를 바꿀 수 있다.**

![img](../assets/%EA%B7%B8%EB%A6%BC%2038-4.png)

이와 같이 축의 순서를 지정하면, 그에 맞게 데이터 축이 달라지며 인수를 `None`(=`x.transpose()`) 이면 **축이 역순으로 정렬되어 행렬이 전치**된다.
이를 반영하기 위해 다음 코드와 같이 수정한다.

```python
class Transpose(Function):
    def __init__(self, axes=None):
        self.axes = axes # 축 정보

    def forward(self, x):
        y = x.transpose(self.axes)
        return y

    def backward(self, gy):
        if self.axes is None:
            # 축 정보가 없는 경우에는 기존 방식대로 행렬 전치
            return transpose(gy)

        axes_len = len(self.axes)
        inv_axes = tuple(np.argsort([ax % axes_len for ax in self.axes]))
        
        return transpose(gy, inv_axes)


def transpose(x, axes=None):
    return Transpose(axes)(x)

```

```python
import dezero
class Variable:
    ...
    
    def transpose(self, *axes):
        if len(axes) == 0:
            axes = None
        elif len(axes) == 1:
            if isinstance(axes[0], (tuple, list)) or axes[0] is None:
                axes = axes[0]
        return dezero.functions.transpose(self, axes)

```

In [5]:
A,B,C,D = 1,2,3,4
x = np.random.rand(1,2,3,4)
y = x.transpose(1,0,3,2)

In [6]:
x = Variable(np.random.rand(1,2,3,4))
print(x)
print("="*50)
y = x.transpose(1,0,3,2)
print(y)

variable([[[[0.54983874 0.18918963 0.21769689 0.52316755]
            [0.70752064 0.50749211 0.50430095 0.88532736]
            [0.39712598 0.37166476 0.59555308 0.10658218]]
         
           [[0.17447957 0.99566373 0.36200139 0.22022387]
            [0.92652781 0.77512771 0.58905322 0.04673174]
            [0.52486712 0.21461378 0.05619512 0.83308894]]]])
variable([[[[0.54983874 0.70752064 0.39712598]
            [0.18918963 0.50749211 0.37166476]
            [0.21769689 0.50430095 0.59555308]
            [0.52316755 0.88532736 0.10658218]]]
         
         
          [[[0.17447957 0.92652781 0.52486712]
            [0.99566373 0.77512771 0.21461378]
            [0.36200139 0.58905322 0.05619512]
            [0.22022387 0.04673174 0.83308894]]]])
